In [1]:
from os import path

from compute_device import PREFERRED_DEVICE

PROJECT_ROOT = path.abspath(path.join(globals()['_dh'][0], '..'))
DATALAKE_PATH = path.abspath(path.join(PROJECT_ROOT, '..', '..', 'datalake', 'llmadapterproj'))
FQFN_PROCESSED_DF = path.join(DATALAKE_PATH, 'processed', f'processed_20250218.jsonl')
MODEL_DIR = path.abspath(path.join(PROJECT_ROOT, 'llmadapterproj', 'classifier'))

2025-02-17 17:05:46,008 - tensorcraft - INFO - XLA Device Not Supported: No module named 'torch_xla'
2025-02-17 17:05:46,018 - tensorcraft - INFO - Pytorch version=2.6.0 preferred device=mps build with MPS support=True


In [2]:
import pandas as pd

df = pd.read_json(f'file://{FQFN_PROCESSED_DF}', orient='records', lines=True)
print(f'df shape={df.shape}')
print(f'df columns={df.columns}')

df shape=(704, 7)
df columns=Index(['file_name', 'fqfn', 'text_body', 'input_ids', 'attention_mask',
       'token_type_ids', 'label'],
      dtype='object')


In [3]:
from llmadapterproj.trainer.llmadapter_trainer import LlmAdapterTrainer, TrainerConf

trainer = LlmAdapterTrainer(df, TrainerConf())

2025-02-17 17:05:47,578 - tensorcraft - INFO - XLA Device Not Supported: No module named 'torch_xla'
2025-02-17 17:05:47,579 - tensorcraft - INFO - Pytorch version=2.6.0 preferred device=mps build with MPS support=True
2025-02-17 17:05:47,583 - tensorcraft - INFO - resolved device_name: mps compute_device: mps tensor_device: mps
Some weights of RobertaAdapterModel were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-02-17 17:05:48,538 - tensorcraft - INFO - LLM Adapter model=microsoft/graphcodebert-base on mps with (#trainable_parameters, #all_parameters)=(127669211, 127669211)


In [4]:
from time import time
start_time = time()

trainer.train()

print(f'Training duration: {time() - start_time:.4f} seconds')

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Roc Auc
1,No log,0.046459,0.992908,0.992990,0.992908,0.992897,0.991071
2,No log,0.001111,1.000000,1.000000,1.000000,1.000000,1.000000
3,No log,0.000221,1.000000,1.000000,1.000000,1.000000,1.000000
4,No log,0.000508,1.000000,1.000000,1.000000,1.000000,1.000000
5,No log,0.000874,1.000000,1.000000,1.000000,1.000000,1.000000
6,No log,0.000614,1.000000,1.000000,1.000000,1.000000,1.000000
7,No log,0.000510,1.000000,1.000000,1.000000,1.000000,1.000000


There were missing keys in the checkpoint model loaded: ['roberta.prompt_tuning.base_model_embeddings.weight', 'heads.default.3.weight'].


Training duration: 384.5013 seconds


In [5]:
# save adapter and tokenizer
trainer.save()

In [6]:
from llmadapterproj.classifier.llmadapter_classifier import load_model_with_adapter
import torch

# Load the fine-tuned model
model, tokenizer = load_model_with_adapter(device_name=PREFERRED_DEVICE)

# New PowerShell script to classify
new_script = """
function Get-SensitiveInfo {
    param($User)
    Get-ADUser -Identity $User -Properties PasswordLastSet, AccountExpirationDate
}
"""

# Tokenize input script and move it the same device as a Model
inputs = tokenizer(new_script, truncation=True, padding='max_length', max_length=512, return_tensors='pt')
inputs = {k: v.to(PREFERRED_DEVICE) for k, v in inputs.items()}

# Predict
model.eval()
with torch.no_grad():
    outputs = model(**inputs)

# Interpret results
logits = outputs.logits
predicted_class = torch.argmax(logits).item()

label_map = {0: 'Benign', 1: 'Malicious'}
print(f'Prediction: {label_map[predicted_class]}')


Some weights of RobertaAdapterModel were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-02-17 17:12:14,334 - tensorcraft - INFO - resolved device_name: mps compute_device: mps tensor_device: mps


Prediction: Malicious
